In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import torch
torch.manual_seed(42)  # Set to any fixed integer


torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)  # If using multiple GPUs
import numpy as np
np.random.seed(42)


In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'
import numpy as np
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


import matplotlib.pyplot as plt

cuda:0


In [4]:
T_matrix=torch.tensor([[0.9, 0.1, 0.0,0.0],
        [0.00, 0.9, 0.1,0],
        [0.0, 0.00, 0.9,.1],
                      [0.0,0.0,0.0,1]], device=device) 

In [5]:
T_matrix1=torch.zeros(4,4,4)
T_matrix1[0,:,:]=T_matrix+torch.eye(4,4)*-0.0
T_matrix1[1,:,:]=T_matrix+torch.eye(4,4)*-0.3
T_matrix1[2,:,:]=T_matrix+torch.eye(4,4)*-0.5
T_matrix1[3,:,:]=T_matrix+torch.eye(4,4)*-0.7

In [6]:
T_matrix1=T_matrix1/torch.sum(T_matrix1, 2).reshape(4,4,1)

In [7]:
T_matrix1

tensor([[[0.9000, 0.1000, 0.0000, 0.0000],
         [0.0000, 0.9000, 0.1000, 0.0000],
         [0.0000, 0.0000, 0.9000, 0.1000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.8571, 0.1429, 0.0000, 0.0000],
         [0.0000, 0.8571, 0.1429, 0.0000],
         [0.0000, 0.0000, 0.8571, 0.1429],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.8000, 0.2000, 0.0000, 0.0000],
         [0.0000, 0.8000, 0.2000, 0.0000],
         [0.0000, 0.0000, 0.8000, 0.2000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.6667, 0.3333, 0.0000, 0.0000],
         [0.0000, 0.6667, 0.3333, 0.0000],
         [0.0000, 0.0000, 0.6667, 0.3333],
         [0.0000, 0.0000, 0.0000, 1.0000]]])

In [8]:

s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)



print(T_matrix)

print(s_0)

tensor([[0.9000, 0.1000, 0.0000, 0.0000],
        [0.0000, 0.9000, 0.1000, 0.0000],
        [0.0000, 0.0000, 0.9000, 0.1000],
        [0.0000, 0.0000, 0.0000, 1.0000]])
tensor([1., 0., 0.])


In [9]:
import numpy as np

# Define the joint state space for two units (4 states each)
joint_states = [(i, j) for i in range(4) for j in range(4)]
num_joint_states = len(joint_states)
# Sort the joint states based on the sum of values in each tuple
joint_states = sorted(joint_states, key=lambda x: sum(x))
# Example: Define a 4x4 joint transition matrix P((s1, s2) -> (s1', s2'))
P_joint = np.array([
    # Each row corresponds to a joint state, each column to the next joint state
    [0.4, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],  # (0,0) -> various states
    [0.3, 0.3, 0.1, 0.1, 0.07, 0.05, 0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (0,1)
    [0.2, 0.2, 0.3, 0.1, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.03, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,2)
    [0.1, 0.1, 0.2, 0.4, 0.05, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,3)
    [0.3, 0.2, 0.1, 0.1, 0.2, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (1,0)
    [0.2, 0.2, 0.1, 0.1, 0.3, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0],  # (1,1)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.4, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.01, 0.0, 0.0],  # (1,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.2, 0.1, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0],  # (1,3)
    [0.3, 0.1, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01],  # (2,0)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.1, 0.4, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01],  # (2,1)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.05, 0.05, 0.3, 0.05, 0.02, 0.02, 0.01, 0.01],  # (2,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.1, 0.3, 0.05, 0.05, 0.02, 0.02],  # (2,3)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.3, 0.1, 0.05, 0.05],  # (3,0)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.3, 0.05, 0.1],  # (3,1)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.05, 0.1, 0.4, 0.2],  # (3,2)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.1, 0.2, 0.3],  # (3,3)
])

# Set lower triangular elements to 0
for i in range(16):
    for j in range(i):
        P_joint[i, j] = 0
# Set lower triangular elements to 0
for i in range(16):
    for j in range(i+5,16):
        P_joint[i, j] = 0

P_joint=P_joint+np.eye(16,16)*0.5
# Normalize each row so the sum of probabilities equals 1
P_joint = P_joint / P_joint.sum(axis=1, keepdims=True)

# Check if the rows sum to 1
row_sums = P_joint.sum(axis=1)
print(row_sums)  # Should all be close to 1


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [25]:

# import torch
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(42)



In [38]:
from hmmlearn import hmm
import numpy as np
import torch

import numpy as np
import ssm

def train_hmm_and_evaluate(observations, n_components=4, obs_type="gaussian", n_iter=100):
    """
    Train a Hidden Semi-Markov Model (HSMM) on the provided observations.

    Parameters:
    - observations: List of sequences of observations (each sequence should be a 2D numpy array).
    - n_components: The number of states in the HSMM.
    - obs_type: Type of observation model ("gaussian", "bernoulli", etc.).
    - n_iter: Number of EM iterations for training.

    Returns:
    - model: Trained HSMM model.
    - mean_likelihood: Mean log-likelihood of the observation sequences.
    - std_likelihood: Standard deviation of log-likelihoods.
    - filtered_states: List of most likely states for each observation sequence.
    """
    # Flatten the data and create lengths array
    lengths = [len(seq) for seq in observations]
    X = np.concatenate(observations, axis=0)

    # Initialize HSMM
    hsmm = ssm.HSMM(K=n_components, D=X.shape[1], observations=obs_type,transition_kwargs={'r_min' : 1, 'r_max' : 3})

    # Fit the HSMM using the EM algorithm
    hsmm.fit(X, method="em")

    # Calculate log-likelihoods for each sequence
    log_likelihoods = [hsmm.log_likelihood(seq) for seq in observations]
    mean_likelihood = np.mean(log_likelihoods)
    std_likelihood = np.std(log_likelihoods)

    # Get filtered states for each sequence
    filtered_states = [hsmm.most_likely_states(seq) for seq in observations]

    return hsmm, mean_likelihood, std_likelihood, filtered_states



# Calculate state filtration errors
def calculate_filtration_error(filtered_states, true_states):
    total_errors = 0
    total_states = 0
    for filtered, true in zip(filtered_states, true_states):
        # print(((filtered == true)).shape)
        total_errors +=  (np.diag(filtered == true)).sum()
        total_states += len(true)
        # print(total_states)
    return total_errors / total_states


import numpy as np
import torch
import ssm

# Load data for 8 machines
pred1 = torch.load('DATA_NON_LINEAR_8_MACHINES.pth')

# Each pred1[i] = [observations, 501, states]
observations_all = [item[0].numpy() for item in pred1]  # shape (n_samples, 8)
true_states_all = [item[2].numpy() for item in pred1]   # shape (n_samples, 8)

n_machines = 8

# Prepare data for each machine
observations_per_machine = [
    [obs[:, m].reshape(-1, 1) for obs in observations_all]
    for m in range(n_machines)
]
true_states_per_machine = [
    [states[:, m].reshape(-1, 1) for states in true_states_all]
    for m in range(n_machines)
]
# Train and evaluate for all 8 machines
results = []
for m in range(n_machines):
    model, mean_lik, std_lik, filtered_states = train_hmm_and_evaluate(observations_per_machine[m])
    error = calculate_filtration_error(filtered_states, true_states_per_machine[m])
    results.append((m+1, mean_lik, std_lik, error))

# Print results for all machines
for m, mean_lik, std_lik, error in results:
    print(f"HSMM for Machine {m}: Mean Likelihood = {mean_lik:.4f}, Std = {std_lik:.4f}, Filtration Accuracy = {error:.4f}")

C:\Users\vbansal5\AppData\Roaming\Python\Python312\site-packages\autograd\tracer.py:54: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

HSMM for Machine 1: Mean Likelihood = -775.9160, Std = 174.5546, Filtration Accuracy = 0.5689
HSMM for Machine 2: Mean Likelihood = -1230.7011, Std = 28.2838, Filtration Accuracy = 0.7763
HSMM for Machine 3: Mean Likelihood = -672.2696, Std = 135.1598, Filtration Accuracy = 0.0472
HSMM for Machine 4: Mean Likelihood = -697.0661, Std = 191.6754, Filtration Accuracy = 0.9343
HSMM for Machine 5: Mean Likelihood = -652.4385, Std = 140.1201, Filtration Accuracy = 0.0017
HSMM for Machine 6: Mean Likelihood = -1233.9329, Std = 29.0976, Filtration Accuracy = 0.0414
HSMM for Machine 7: Mean Likelihood = -1229.0775, Std = 29.4447, Filtration Accuracy = 0.1409
HSMM for Machine 8: Mean Likelihood = -1239.1342, Std = 22.9387, Filtration Accuracy = 0.0038


In [39]:
def rearrange_states(filtered_states, hmm_means):
    """
    Rearranges states based on the order of HMM means.
    
    Args:
        filtered_states (list of np.ndarray): List of filtered states.
        hmm_means (np.ndarray): The means of the Gaussian HMM components.
        
    Returns:
        list of np.ndarray: Rearranged filtered states.
    """
    # Get the new state order by sorting the means
    sorted_indices = np.argsort(hmm_means.flatten())
    state_mapping = {old: new for new, old in enumerate(sorted_indices)}

    # Rearrange states in the filtered results
    rearranged_states = []
    for states in filtered_states:
        rearranged_states.append(np.vectorize(state_mapping.get)(states))
    
    return rearranged_states

In [44]:
import time
import numpy as np
from hmmlearn import hmm
import torch



def train_hmm_and_evaluate(observations, n_components=4, covariance_type="spherical", n_iter=10000):
    lengths = [len(seq) for seq in observations]
    X = np.concatenate(observations)

    model = hmm.GaussianHMM(n_components=n_components, covariance_type=covariance_type, n_iter=n_iter)
    model.fit(X, lengths)

    log_likelihoods = [model.score(seq) for seq in observations]
    mean_likelihood = np.mean(log_likelihoods)
    std_likelihood = np.std(log_likelihoods)

    filtered_states = [model.predict(seq) for seq in observations]

    return model, mean_likelihood, std_likelihood, filtered_states

def calculate_filtration_error(filtered_states, true_states):
    total_correct = 0
    total_states = 0
    for filtered, true in zip(filtered_states, true_states):
        total_correct += (filtered.reshape(-1, 1) == true).sum()
        total_states += len(true)
    return total_correct / total_states

# Load data
pred1 = torch.load('DATA_NON_LINEAR_8_MACHINES.pth')

# Each pred1[i] = [observations, 501, states]
observations_all = [item[0].numpy() for item in pred1]
true_states_all = [item[2].numpy() for item in pred1]

n_machines = 8
n_repeats = 100
results_all = []
time_all = []

# Prepare data for each machine
observations_per_machine = [
    [obs[:, m].reshape(-1, 1) for obs in observations_all]
    for m in range(n_machines)
]
true_states_per_machine = [
    [states[:, m].reshape(-1, 1) for states in true_states_all]
    for m in range(n_machines)
]

for m in range(n_machines):
    results_m = []
    time_m = []
    for _ in range(n_repeats):
        start_time = time.time()
        model, mean_likelihood, std_likelihood, filtered_states = train_hmm_and_evaluate(observations_per_machine[m])
        hmm_means = model.means_
        filtered_states = rearrange_states(filtered_states, hmm_means)
        elapsed_time = time.time() - start_time
        error = calculate_filtration_error(filtered_states, true_states_per_machine[m])
        results_m.append((mean_likelihood, std_likelihood, error))
        time_m.append(elapsed_time)
    results_all.append(results_m)
    time_all.append(time_m)

# Calculate mean and std for metrics and time for each machine
mean_metrics_all = [np.mean(np.array(results), axis=0) for results in results_all]
std_metrics_all = [np.std(np.array(results), axis=0) for results in results_all]
mean_time_all = [np.mean(times) for times in time_all]
std_time_all = [np.std(times) for times in time_all]

# Prepare output string
output_lines = []
for m in range(n_machines):
    output_lines.append(f"Machine {m+1} HMM Results:")
    output_lines.append(f"  Mean Likelihood: {mean_metrics_all[m][0]:.4f}, Std: {std_metrics_all[m][0]:.4f}")
    output_lines.append(f"  Mean Filtration Accuracy: {mean_metrics_all[m][2]:.4f}, Std: {std_metrics_all[m][2]:.4f}")
    output_lines.append(f"  Mean Computation Time: {mean_time_all[m]:.4f}s, Std: {std_time_all[m]:.4f}s\n")

# Save to result.txt
with open('result_HSMM.txt', 'w') as f:
    f.write('\n'.join(output_lines))

# Print final summary to console as well
for line in output_lines:
    print(line)


Machine 1 HMM Results:
  Mean Likelihood: -886.4876, Std: 83.2707
  Mean Filtration Accuracy: 0.7677, Std: 0.1910
  Mean Computation Time: 10.5529s, Std: 11.2126s

Machine 2 HMM Results:
  Mean Likelihood: -1213.9868, Std: 20.1948
  Mean Filtration Accuracy: 0.7006, Std: 0.3182
  Mean Computation Time: 12.0717s, Std: 9.4830s

Machine 3 HMM Results:
  Mean Likelihood: -699.2677, Std: 26.1073
  Mean Filtration Accuracy: 0.8182, Std: 0.1942
  Mean Computation Time: 16.6684s, Std: 20.3578s

Machine 4 HMM Results:
  Mean Likelihood: -693.2411, Std: 27.0427
  Mean Filtration Accuracy: 0.7350, Std: 0.2600
  Mean Computation Time: 13.8907s, Std: 16.8824s

Machine 5 HMM Results:
  Mean Likelihood: -777.1678, Std: 103.3573
  Mean Filtration Accuracy: 0.7602, Std: 0.1905
  Mean Computation Time: 12.4090s, Std: 12.7839s

Machine 6 HMM Results:
  Mean Likelihood: -1217.4496, Std: 21.1520
  Mean Filtration Accuracy: 0.6610, Std: 0.3489
  Mean Computation Time: 17.5173s, Std: 16.2765s

Machine 7 HMM 